In [ ]:
import pandas as pd
from pathlib import Path

# Tutorial for handling FMU models 
The aim of this tutorial is to demonstrate how to load FMU models on python using <code>ModelicaFmuModel</code> class.

# 1. Proposed model 

In this tutorial, we load and run an FMU model created with Python beforehand.

The model is a uses a resistance-capacity approach (4R2C) of a wall installed in a test bench, with : 
- inside-outside convection/conduction transfers
- Infrared transfers with the sky and surrounding
- Short wave solar radiation transfers
- External and internal temperature as boundary conditions.

The model was built using openModelica then exported as an FMU: 

<img src="images/OM_eticsmodel.png"  height="200">


# 2. Set boundary file
First, let us load measurement data on python, which will be used as our boundary conditions

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/study_df.csv",
    index_col=0,
    parse_dates=True
)    

# 2. Set simulations options

The used class for running the FMU model requires a model path, simulation options, and optionaly, a reference dataframe for boundary options (to override the default one) and a list of outputs.

We already loaded the boundary file. We can set the simulation options:
- Start time and stop time should be in second. We can use the index of the <code>DataFrame</code> we just created.
The modelitool function <code>modelitool.combitabconvert.datetime_to_seconds</code>
helps you convert datetime index in seconds.
- The solver in the simulation options must be one of 'Euler' or 'CVode'.
- The output interval is in seconds.

In [ ]:
from modelitool.combitabconvert import datetime_to_seconds

In [ ]:
second_index = datetime_to_seconds(reference_df.index)

In [ ]:
simulation_options_FMU = {
    "startTime":second_index[0],
    "stopTime": second_index[-1],
    "solver": "CVode",  
    "outputInterval": 300,
}

In [ ]:
from modelitool.combitabconvert import df_to_combitimetable

In [ ]:
df_to_combitimetable(
    df=reference_df.loc["2018-03-22":"2018-03-23"],
    filename="resources/boundary_temp.txt"
)

Finally, we can define a list of output that will be included in the dataframe output for any simulation.

In [ ]:
output_list  = [
    "T_coat_ins.T",
     "T_ins_ins.T",
     "Tw_out.T"
]

#  3. Instantiate ModelicaFmuModel

Now, we can also load an FMU <code>ModelicaModel</code> from <code>corrai.fmu</code>

In [ ]:
from corrai.fmu import ModelicaFmuModel 

Set the initial and parameter values in a dictionary.

In [ ]:
parameter_dict_OM = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
    'Lambda_ins.k': 0.04,
}

Simulation flags can be specified in <code>simulate()</code> method. Overview of possible simulation flags can be found here: https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/simulationflags.html. Note that the simulation flag <code>override</code> cannot be used, as it was already used in class <code>OMModel</code> with <code>simulation_options</code>.

If x boundary conditions is not specified or do not
    have a DateTime index (seconds int), a year can be specified to convert
    int seconds index to a datetime index. If simulation spans overs several
    years, it shall be the year when it begins.

In [ ]:
init_res_OM = simu_OM.simulate(
    simflags = "-initialStepSize=60 -maxStepSize=3600 -w -lv=LOG_STATS",
    parameter_dict=parameter_dict_OM,
    year=2024,
)

Results are displayed in a dataframe:

In [ ]:
init_res_OM

Plotted results

In [ ]:
init_res_OM.plot()

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

simu_FMU = ModelicaFmuModel(
    fmu_path=Path(TUTORIAL_DIR) / "resources/etics_v0.fmu",
    simulation_options=simulation_options_FMU,
    x = reference_df,
    output_list=output_list,
)

#  4. Run a simulation

Set the initial and parameter values in a dictionary.

In [ ]:
parameter_dict = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
    'Lambda_ins.k': 0.04,
}

And run a simulation. To enable the display of FMU debug logging, set <code>debug_logging</code> to True.

In [ ]:
init_res_FMU = simu_FMU.simulate(
    parameter_dict = parameter_dict,
    debug_logging=False
)

Results are displayed in a dataframe:

In [ ]:
init_res_FMU

Plotted results

In [ ]:
init_res_FMU.plot()